In [ ]:
import pandas as pd

# try diagnosing from say 6 to 4 or something my indexing seems to work as it should
# so i think the issue is when the centroid brings back a group not every row may truly belong to that group
# Could there be some kind of test to see if ever row agrees to the new cluster ? or they will switch teams.
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


df_ir = pd.read_csv('/content/drive/My Drive/dsc_dir/IRIS.csv')
df_ir1 = df_ir.drop('species', axis=1)
# # Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler to the data and transform it
scaled_data = scaler.fit_transform(df_ir1)

# Convert the scaled data back into a pandas DataFrame
df_iris = pd.DataFrame(scaled_data, columns=df_ir1.columns)
# df_iris = df_iris.sample(frac=1).reset_index(drop=True)
# df_iris_sample = df_iris.sample(n=40).reset_index(drop=True)


# MAIN MODEL

## Requirements

1. USE ROBUST SCALING LIKE MIN MAX AND ROBOST SCALER FOR DATA

If the scaling method you used is sensitive to outliers (like Z-score normalization), you might benefit from rescaling after outlier removal. However, if you used a more robust scaling method (like Min-Max scaling or Robust Scaler in sklearn that scales data according to percentiles and is thus less sensitive to outliers), rescaling might not be as necessary.


## Outlier removal

In [ ]:
# import pandas as pd
# import numpy as np
# df11 = pd.read_hdf('/content/drive/My Drive/dsc_dir/Godiva_df_emb.h5', key='df11')
# embeddings_array = np.stack(df11["gpt_3_large_emb"].values)

# Array with loop and outlier removal


In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances

# Assuming embeddings is a numpy array already defined
df_original = df_14.values.copy()

# Compute the pairwise distances
dist_matrix = pairwise_distances(df_original, metric='cosine')

# Replace diagonal with np.inf to ignore self-distance
np.fill_diagonal(dist_matrix, np.inf)

# Find the index of the closest row and the distance for each row
closest_indices = np.argmin(dist_matrix, axis=1)
closest_distances = np.min(dist_matrix, axis=1)

def remove_upper_outliers_and_save(data, distances):
    quartile_1, quartile_3 = np.percentile(distances, [25, 75])
    iqr = quartile_3 - quartile_1
    upper_bound = quartile_3 + (iqr * 1.5)

    not_outliers_mask = distances <= upper_bound
    outliers_mask = distances > upper_bound

    filtered_data = data[not_outliers_mask]
    outliers_data = data[outliers_mask]

    return filtered_data, outliers_data

filtered_data, outliers_data = remove_upper_outliers_and_save(df_original, closest_distances)

# If you simply want to track the indices, you could just keep them as separate arrays:
filtered_ids = np.array(['unq_id_' + str(i) for i in range(filtered_data.shape[0])])
outlier_ids = np.array(['unq_id_' + str(i) for i in range(filtered_data.shape[0], filtered_data.shape[0] + outliers_data.shape[0])])

def remove_upper_outliers_and_save(distances):
    quartile_1, quartile_3 = np.percentile(distances, [25, 75])
    iqr = quartile_3 - quartile_1
    upper_bound = quartile_3 + 1.5 * iqr
    not_outliers_mask = distances <= upper_bound
    return not_outliers_mask

not_outliers_mask = remove_upper_outliers_and_save(closest_distances)
filtered_df11 = df11.iloc[not_outliers_mask].copy(deep=True)


In [ ]:
%%time

import networkx as nx
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import numpy as np

# Assuming df_original is already defined

df_original = filtered_data.copy()
# df_original['fam_id'] = ['unq_id_' + str(i) for i in range(len(df_original))]

fam_ids = np.array(['unq_id_' + str(i) for i in range(len(df_original))])
fam_ids_array = np.array(fam_ids).reshape(-1, 1)
new_array = np.concatenate((df_original, fam_ids_array), axis=1)

# 2. MAIN NETWORK CLUSTERING

## 2A Find closet distance and index of rows with closes distance in a different fam_id
# Assuming df is your DataFrame and it includes 'fam_id' along with the features.

# Compute the pairwise distances using only the feature columns.
dist_matrix0 = pairwise_distances(df_original, metric='cosine')



In [ ]:
breaker = 32

In [ ]:
%%time
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# ee
iteration_list = []
sil_perf_list = []
calinski_perf_list = []
unique_fam_list = []


# NUCER

for ir in range(50):
    dist_matrix = np.copy(dist_matrix0)
    fam_ids = new_array[:, -1]
    same_fam_id = new_array[:, -1][:, None] == new_array[:, -1]
    dist_matrix[same_fam_id] = np.inf

    closest_indices = np.argmin(dist_matrix, axis=1)
    closest_distances = np.min(dist_matrix, axis=1)

    # Handling migrations
    closest_families = fam_ids[closest_indices]
    dtype = [('fam_id', 'U10'), ('closest_distance', 'f8'), ('closest_family', 'U10')]
    structured_array = np.array(list(zip(fam_ids, closest_distances, closest_families)), dtype=dtype)
    sorted_structured_array = np.sort(structured_array, order=['fam_id', 'closest_distance'])
    _, unique_indices = np.unique(sorted_structured_array['fam_id'], return_index=True)
    result_structured_array = sorted_structured_array[unique_indices]
    migration_map = {fam_id: closest_family for fam_id, closest_family in result_structured_array[['fam_id', 'closest_family']]}
    migrate_to_family = np.array([migration_map[fam] for fam in fam_ids])


    # Assuming fam_ids, migrate_to_families, and closest_distances are defined as NumPy arrays

    # Create a structured array for sorting and unique operation
    dtype = [('migrate_to_family', 'U10'), ('closest_distance', float), ('fam_id', 'U10')]
    structured_array = np.array(list(zip(migrate_to_family , closest_distances, fam_ids)), dtype=dtype)

    # Sort by 'migrate_to_family' and 'closest_distance'
    sorted_array = np.sort(structured_array, order=['migrate_to_family', 'closest_distance'])

    # Remove duplicates
    _, unique_indices = np.unique(sorted_array['migrate_to_family'], return_index=True)
    unique_families = sorted_array[unique_indices]

    # Mapping for migration decisions
    migration_decision_map = {entry['migrate_to_family']: entry['fam_id'] for entry in unique_families}

    # Calculate the minimum distance per migration family
    unique_families_distances = {entry['migrate_to_family']: entry['closest_distance'] for entry in unique_families}

    # Assign migration permissions
    can_migrate = np.array([closest_distances[i] == unique_families_distances[migrate_to_family[i]] for i in range(len(fam_ids))])

    import networkx as nx

    uf = nx.utils.UnionFind()
    for i, entry in enumerate(structured_array):
        uf.union(entry['fam_id'], entry['migrate_to_family'])

    # Map the results back
    fam_id_mapping = {item: uf[item] for item in uf.parents.keys()}
    mapped_fam_ids = np.array([fam_id_mapping[fam_id] for fam_id in fam_ids])

    from sklearn.metrics import pairwise_distances

    # Calculate centroids
    unique_fam_ids = np.unique(mapped_fam_ids)
    centroids = np.array([df_original[mapped_fam_ids == fam_id].mean(axis=0) for fam_id in unique_fam_ids])

    # Calculate distances from each point to each centroid and find the closest
    distances = pairwise_distances(df_original, centroids, metric='cosine')
    closest_centroids = np.argmin(distances, axis=1)

    # Assign new fam_ids based on the closest centroid
    new_fam_ids = np.array([unique_fam_ids[idx] for idx in closest_centroids])
    fam_ids_array1 = np.array(new_fam_ids).reshape(-1, 1)
    new_array = np.concatenate((df_original, fam_ids_array1), axis=1)


    features = new_array[:, :-1].astype(float)
    # features = new_array[:, :-1]  # All rows, all columns except the last

    # Extract 'fam_id' for labels
    labels = new_array[:, -1]  # Last column for labels




    sil_perf = silhouette_score(features, labels)
    calinski_harabasz = calinski_harabasz_score(features, labels)
    sil_perf_list.append(sil_perf)
    calinski_perf_list.append(calinski_harabasz)
    unique_fam_list.append(len(set(new_fam_ids)))
    iteration_list.append(ir)


    if len(set(new_fam_ids)) == breaker:
        break

data = {
    'iteration_list': iteration_list,
    'sil_perf_list': sil_perf_list,
    'calinski_perf_list': calinski_perf_list,
    'unique_fam_list': unique_fam_list
}

# Create DataFrame
df_eval = pd.DataFrame(data)

df_eval


In [ ]:
len(set(new_fam_ids))

In [ ]:

filtered_df11['label'] = new_fam_ids
outliers_mask = ~not_outliers_mask  # Get the outliers mask
outliers_df11 = df11.iloc[outliers_mask].copy(deep=True)  # Extract outliers
outliers_df11['label'] = -1
df_comb = pd.concat([filtered_df11, outliers_df11], axis=0)


In [ ]:
df21 = df_comb.drop(['gpt_3_large_emb'],axis=1)
df21.to_excel('Godiva_done_32_pca.xlsx')


# Array no loop no outlier

In [ ]:
%%time

import networkx as nx
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import numpy as np

# Assuming df_original is already defined

df_original = df_iris.values.copy()
# df_original['fam_id'] = ['unq_id_' + str(i) for i in range(len(df_original))]

fam_ids = np.array(['unq_id_' + str(i) for i in range(len(df_original))])
fam_ids_array = np.array(fam_ids).reshape(-1, 1)
new_array = np.concatenate((df_original, fam_ids_array), axis=1)

# 2. MAIN NETWORK CLUSTERING

## 2A Find closet distance and index of rows with closes distance in a different fam_id
# Assuming df is your DataFrame and it includes 'fam_id' along with the features.

# Compute the pairwise distances using only the feature columns.
dist_matrix0 = pairwise_distances(df_original, metric='euclidean')



In [ ]:
# Finding the closest valid entries

dist_matrix = np.copy(dist_matrix0)
fam_ids = new_array[:, -1]
same_fam_id = new_array[:, -1][:, None] == new_array[:, -1]
dist_matrix[same_fam_id] = np.inf

closest_indices = np.argmin(dist_matrix, axis=1)
closest_distances = np.min(dist_matrix, axis=1)

# Handling migrations
closest_families = fam_ids[closest_indices]
dtype = [('fam_id', 'U10'), ('closest_distance', 'f8'), ('closest_family', 'U10')]
structured_array = np.array(list(zip(fam_ids, closest_distances, closest_families)), dtype=dtype)
sorted_structured_array = np.sort(structured_array, order=['fam_id', 'closest_distance'])
_, unique_indices = np.unique(sorted_structured_array['fam_id'], return_index=True)
result_structured_array = sorted_structured_array[unique_indices]
migration_map = {fam_id: closest_family for fam_id, closest_family in result_structured_array[['fam_id', 'closest_family']]}
migrate_to_family = np.array([migration_map[fam] for fam in fam_ids])


# Assuming fam_ids, migrate_to_families, and closest_distances are defined as NumPy arrays

# Create a structured array for sorting and unique operation
dtype = [('migrate_to_family', 'U10'), ('closest_distance', float), ('fam_id', 'U10')]
structured_array = np.array(list(zip(migrate_to_family , closest_distances, fam_ids)), dtype=dtype)

# Sort by 'migrate_to_family' and 'closest_distance'
sorted_array = np.sort(structured_array, order=['migrate_to_family', 'closest_distance'])

# Remove duplicates
_, unique_indices = np.unique(sorted_array['migrate_to_family'], return_index=True)
unique_families = sorted_array[unique_indices]

# Mapping for migration decisions
migration_decision_map = {entry['migrate_to_family']: entry['fam_id'] for entry in unique_families}

# Calculate the minimum distance per migration family
unique_families_distances = {entry['migrate_to_family']: entry['closest_distance'] for entry in unique_families}

# Assign migration permissions
can_migrate = np.array([closest_distances[i] == unique_families_distances[migrate_to_family[i]] for i in range(len(fam_ids))])

import networkx as nx

uf = nx.utils.UnionFind()
for i, entry in enumerate(structured_array):
    uf.union(entry['fam_id'], entry['migrate_to_family'])

# Map the results back
fam_id_mapping = {item: uf[item] for item in uf.parents.keys()}
mapped_fam_ids = np.array([fam_id_mapping[fam_id] for fam_id in fam_ids])

from sklearn.metrics import pairwise_distances

# Calculate centroids
unique_fam_ids = np.unique(mapped_fam_ids)
centroids = np.array([df_original[mapped_fam_ids == fam_id].mean(axis=0) for fam_id in unique_fam_ids])

# Calculate distances from each point to each centroid and find the closest
distances = pairwise_distances(df_original, centroids, metric='euclidean')
closest_centroids = np.argmin(distances, axis=1)

# Assign new fam_ids based on the closest centroid
new_fam_ids = np.array([unique_fam_ids[idx] for idx in closest_centroids])
fam_ids_array1 = np.array(new_fam_ids).reshape(-1, 1)
new_array = np.concatenate((df_original, fam_ids_array1), axis=1)



In [ ]:
len(set(new_fam_ids))

In [ ]:
%%time

    # NUCER
    # 2. MAIN NETWORK CLUSTERING

    ## 2A Find closet distance and index of rows with closes distance in a different fam_id
    # dist_matrix = dist_matrix0.copy()
    dist_matrix = np.copy(dist_matrix0)
    # Reset the index of the DataFrame to keep track of original indices

    # OPTIMIZATION 1
    #############
    # WAS
    # # Get a boolean matrix where True represents rows with the same 'fam_id'.
    # same_fam_id = df_original['fam_id'].values[:, None] == df_original['fam_id'].values

    # # Set the distances for rows with the same 'fam_id' to infinity.
    # # np.inf is used here to ensure these distances are never considered as the minimum.
    # dist_matrix[same_fam_id] = np.inf

    # NOW..SIGNIFICANTLY FASTER
    # df_with_index = df_original.reset_index()

    # Perform the merge to find all pairs with the same 'fam_id'
    # same_fam_id = df_with_index.merge(
    #     df_with_index[['index', 'fam_id']], on='fam_id', suffixes=('_left', '_right'))

    # # Minimal column merge to reduce memory footprint
    # same_fam_id = df_with_index[['index', 'fam_id']].merge(
    #     df_with_index[['index', 'fam_id']], on='fam_id', suffixes=('_left', '_right'))
    # Get a boolean matrix where True represents rows with the same 'fam_id'.
    same_fam_id = df_original['fam_id'].values[:, None] == df_original['fam_id'].values

    # Set the distances for rows with the same 'fam_id' to infinity.
    # np.inf is used here to ensure these distances are never considered as the minimum.
    dist_matrix[same_fam_id] = np.inf


    ##########

    # num_rows = len(df_original)
    # # Initialize a boolean matrix of size (num_rows, num_rows)
    # same_fam_id_matrix = np.zeros((num_rows, num_rows), dtype=bool)

    # # Fill the boolean matrix with True where indices have the same 'fam_id'
    # same_fam_id_matrix[same_fam_id['index_left'].values, same_fam_id['index_right'].values] = True

    # dist_matrix[same_fam_id_matrix]= np.inf
    ##########

    # Find the index of the closest row and the distance for each row, ignoring those with the same 'fam_id'.
    closest_indices = dist_matrix.argmin(axis=1)
    closest_distances = dist_matrix.min(axis=1)

    # Add the results back to the original DataFrame.
    df_original['closest_row'] = closest_indices
    df_original['closest_distance'] = closest_distances


    # OPTIMIZATION 2
    #############
    # WAS

    # Assuming df_original is your original DataFrame and it includes the 'fam_id' column.

    # 2B set up families to migrate to and solve conflicts if mulitple row in the same family want to migrate to the different families
    # Add a column for the family that the closest row belongs to for easier reference
    # df_original['closest_family'] = df_original['closest_row'].apply(lambda x: df_original.at[x, 'fam_id'] if pd.notnull(x) else None)

    # # Determine the closest family for each original family based on the smallest distance
    # closest_family_for_group = df_original.groupby('fam_id').apply(lambda x: x.loc[x['closest_distance'].idxmin(), 'closest_family'])

    # # Map the target family for migration back to the original DataFrame
    # df_original['migrate_to_family'] = df_original['fam_id'].map(closest_family_for_group)

    # NOW

    df_original['closest_family'] = df_original.loc[df_original['closest_row'], 'fam_id'].values

    # Sort the DataFrame first by 'fam_id', then by 'closest_distance'
    sorted_df = df_original.sort_values(by=['fam_id', 'closest_distance'])

    # Drop duplicates, keeping the first entry for each 'fam_id'
    # which will be the one with the smallest 'closest_distance' due to the sort order
    result_df = sorted_df.drop_duplicates(subset='fam_id', keep='first')

    # Select only the 'fam_id' and 'closest_family' columns if necessary
    closest_family_for_group = result_df.set_index('fam_id')['closest_family']

    # Map the target family for migration back to the original DataFrame
    df_original['migrate_to_family'] = df_original['fam_id'].map(closest_family_for_group)
    ##################

    # OPTIMIZATION 3
    #############

    # WAS
    # Choosing only the closest family on the migrate_to_family side
    # Step 1: Determine the eligible family for each target family

    # Group by 'migrate_to_family' and find the family with the closest distance for each target
    # eligible_families = df_original.groupby('migrate_to_family').apply(
    #     lambda x: x.loc[x['closest_distance'].idxmin(), 'fam_id']
    # ).reset_index().rename(columns={0: 'eligible_fam_id'})

    # NOW
    # Sort the DataFrame by 'migrate_to_family' and 'closest_distance'
    sorted_df = df_original.sort_values(by=['migrate_to_family', 'closest_distance'])

    # Drop duplicates, keeping the first entry for each 'migrate_to_family'
    # which will be the one with the smallest 'closest_distance' due to the sort order
    unique_families = sorted_df.drop_duplicates(subset='migrate_to_family', keep='first')

    # Rename and clean up the result if necessary
    eligible_families = unique_families.rename(columns={'fam_id': 'eligible_fam_id'})[['migrate_to_family', 'eligible_fam_id']]

    ##############

    # Step 2: Assign migration permission based on eligibility
    # Merge the eligible families DataFrame back to the original DataFrame to assign migration permission
    df_original = df_original.merge(
        eligible_families,
        how='left',
        left_on=['migrate_to_family', 'fam_id'],
        right_on=['migrate_to_family', 'eligible_fam_id']
    )


    # Step 1 & 2: Group by 'migrate_to_family' and find all families with the smallest distance for each target
    # This involves creating a temporary DataFrame to calculate minimum distances for each migration target group
    min_distances = df_original.groupby('migrate_to_family')['closest_distance'].transform('min')

    # Step 3: Assign migration permission based on matching the minimum distance
    # Families that match the minimum distance to their target are allowed to migrate
    df_original['can_migrate'] = df_original['closest_distance'] == min_distances

    # Step 1: Find all fam_id values where at least one row has can_migrate = True
    fam_ids_to_update = df_original[df_original['can_migrate'] == True]['fam_id'].unique()

    # Step 2: Update can_migrate for rows where fam_id matches the identified fam_ids
    df_original.loc[df_original['fam_id'].isin(fam_ids_to_update), 'can_migrate'] = True

    # Logic to update 'migrate_to_family' where 'can_migrate' is False
    df_original.loc[df_original['can_migrate'] == False, 'migrate_to_family'] = df_original['fam_id']

    # Drop the 'eligible_fam_id' column as it's no longer needed
    df_original.drop('eligible_fam_id', axis=1, inplace=True)

    # Initialize NetworkX UnionFind



    ## OPTIMIZATION 4
    ##################################

    # WAS
    # # Using DataFrame.apply() to perform Union operations on all rows at once
    # uf = nx.utils.UnionFind()
    # df_original.apply(lambda row: uf.union(row['fam_id'], row['migrate_to_family']), axis=1)

    # # Mapping the results back to the DataFrame can be optimized by
    # # creating a mapping dictionary from the UnionFind structure and then using map()
    # fam_id_mapping = {item: uf[item] for item in uf.parents.keys()}

    # df_original['fam_id'] = df_original['fam_id'].map(fam_id_mapping)

    # Drop unnecessary columns in a more concise way
    # columns_to_drop = ['closest_row', 'closest_distance', 'closest_family', 'migrate_to_family', 'can_migrate']
    # df_original.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    # NOW
    # Iterate over DataFrame rows using itertuples() for better performance
    uf = nx.utils.UnionFind()
    for row in df_original.itertuples():
        uf.union(row.fam_id, row.migrate_to_family)

    # Creating a mapping dictionary from the UnionFind structure
    fam_id_mapping = {item: uf[item] for item in uf.parents.keys()}

    # Ensure that all original IDs are covered in the mapping
    df_original['fam_id'] = df_original['fam_id'].map(fam_id_mapping).fillna(df_original['fam_id'])

    # Drop unnecessary columns in a more concise way
    columns_to_drop = ['closest_row', 'closest_distance', 'closest_family', 'migrate_to_family', 'can_migrate']
    df_original.drop(columns=columns_to_drop, inplace=True, errors='ignore')

        # # Solution to all my issues.. recenter
    def calculate_centroids(df):
            """Calculate centroids of families in the dataframe."""
            centroids = df.groupby('fam_id').mean()
            return centroids

    def reassign_families(df, centroids):
        """Reassign rows to the closest family based on Euclidean distance to centroids."""
        # Calculate distances between each row and each centroid
        distances = pairwise_distances(df.drop('fam_id',axis=1), centroids, metric='euclidean')

        # Find the closest centroid for each row
        closest_centroids = np.argmin(distances, axis=1)

        # Map centroid indices back to family labels
        index_to_family = {i: family for i, family in enumerate(centroids.index)}
        df['fam_id'] = [index_to_family[index] for index in closest_centroids]

        return df

    # # Calculate initial centroids
    centroids = calculate_centroids(df_original)

    # Reassign families based on closest centroid
    df_original = reassign_families(df_original, centroids)

    ####################################

In [ ]:
df_original.fam_id.nunique()

In [ ]:
# # Medoid

# # Reset the index to ensure alignment
# df_with_index = df_original.reset_index()
# # Perform the merge on 'fam_id' to identify the same family pairings
# same_fam_id = df_with_index.merge(df_with_index[['index', 'fam_id']],
#                     on='fam_id', suffixes=('_left', '_right'))

# # Initialize a  full infinity matrix
# intra_fam_dist_matrix = np.full((len(df_original), len(df_original)), np.inf)

# # Fill only the distances for the same family members
# intra_fam_dist_matrix[same_fam_id['index_left'].values, same_fam_id['index_right'].values] = dist_matrix0[same_fam_id['index_left'].values, same_fam_id['index_right'].values]

# # Sum distances in the intra-family distance matrix
# sum_distances = np.sum(intra_fam_dist_matrix, axis=1)

# # Add a column to df_original to identify the medoid's index within each family
# df_with_index['sum_distances'] = sum_distances

# # Identify the index with the minimum sum of distances within each family group
# medoids = df_with_index.loc[df_with_index.groupby('fam_id')['sum_distances'].idxmin()]

# medoid_family_mapping = df_original.loc[medoids.index, ['fam_id']]
# medoid_family_mapping['medoid_index'] = medoids.index
# # Select only the columns of the distance matrix corresponding to the medoids
# medoid_distances = dist_matrix0[:, medoid_family_mapping['medoid_index']]
# closest_medoid_indices = np.argmin(medoid_distances, axis=1)
# # Create a mapping from index in the medoid_distances array to family ID
# index_to_family_id = medoid_family_mapping['fam_id'].iloc[closest_medoid_indices].values
# df_original['fam_id'] = index_to_family_id

# Dataframe version with loop and outlier

In [12]:
%%time

breaker = 2
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
# df_original = df_iris1.copy(deep=True)
#df_original = df_credit_card_3.copy(deep=True)
all_outliers = pd.DataFrame()

all_df_original = pd.DataFrame()
df_original = df_14.copy(deep=True)

## Outlier removal hashed out
# # Compute the pairwise distances
# dist_matrix = pairwise_distances(df, metric='euclidean')

# # Replace diagonal with np.inf to ignore self-distance
# np.fill_diagonal(dist_matrix, np.inf)

# # Find the index of the closest row and the distance for each row
# closest_indices = dist_matrix.argmin(axis=1)
# closest_distances = dist_matrix.min(axis=1)

# # Add the results to the original DataFrame
# df['Closest_Row'] = closest_indices
# df['Distance'] = closest_distances

# # 1. Remove outliers after
# # We will modify the function to also return the outliers

# def remove_upper_outliers_and_save(df, column_name):
#     quartile_1, quartile_3 = np.percentile(df[column_name], [25, 75])
#     iqr = quartile_3 - quartile_1
#     upper_bound = quartile_3 + (iqr * 1.5) # 1.5

#     # Filter the DataFrame to remove data points above the upper bound
#     filtered_df = df[df[column_name] <= upper_bound]
#     # Save the outliers in a separate DataFrame
#     outliers_df = df[df[column_name] > upper_bound]

#     return filtered_df, outliers_df

# # Apply the modified function to remove upper outliers from 'Distance' column and save the removed outliers
# filtered_df, outliers_df = remove_upper_outliers_and_save(df, 'Distance')

# # Now we have two DataFrames: `filtered_df` without outliers and `outliers_df` with only the removed outliers
# # (filtered_df, outliers_df)

# outliers_df0 = outliers_df.copy(deep=True)
# del(outliers_df)
# #filtered_df.reset_index(inplace=True, drop=True)

# df_original = filtered_df.copy(deep=True)
# df_original.drop(['Closest_Row','Distance'], axis = 1, inplace=True)
# outliers_df0.drop(['Closest_Row','Distance'], axis = 1, inplace=True)
# outliers_df0['fam_id'] = -1
# all_outliers = pd.concat([all_outliers, outliers_df0], ignore_index=True)
# # del(outliers_df0)

# df_clean1 = df_cleaned.loc[df_original.index].copy(deep=True)
# df_clean1.reset_index(inplace=True, drop=True)

# # df_original['fam_id'] = ['unq_id_' + str(i) for i in range(len(df_original))]
# df_original.reset_index(drop=True, inplace=True)
#
#
#
# Using factorize to encode 'fam_id'
# df_original['fam_id'] = pd.factorize(df_original['fam_id'])[0]

# # Convert the encoded numbers to strings
# df_original['fam_id'] = df_original['fam_id'].astype(str)




#
#
#
#
#44
# df_original = df_iris.copy(deep=True)
df_original['fam_id'] = ['unq_id_' + str(i) for i in range(len(df_original))]

# 2. MAIN NETWORK CLUSTERING

## 2A Find closet distance and index of rows with closes distance in a different fam_id
import networkx as nx
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
# Assuming df is your DataFrame and it includes 'fam_id' along with the features.

# Compute the pairwise distances using only the feature columns.
features = df_original.drop('fam_id', axis=1)
dist_matrix0 = pairwise_distances(features, metric='euclidean')


# # Solution to all my issues.. recenter
def calculate_centroids(df):
        """Calculate centroids of families in the dataframe."""
        centroids = df.groupby('fam_id').mean()
        return centroids

def reassign_families(df, centroids):
    """Reassign rows to the closest family based on Euclidean distance to centroids."""
    # Calculate distances between each row and each centroid
    distances = pairwise_distances(df.drop('fam_id',axis=1), centroids, metric='euclidean')

    # Find the closest centroid for each row
    closest_centroids = np.argmin(distances, axis=1)

    # Map centroid indices back to family labels
    index_to_family = {i: family for i, family in enumerate(centroids.index)}
    df['fam_id'] = [index_to_family[index] for index in closest_centroids]

    return df

iteration_list = []
sil_perf_list = []
calinski_perf_list = []
unique_fam_list = []


# NUCER

for ir in range(50):

    # 2. MAIN NETWORK CLUSTERING

    ## 2A Find closet distance and index of rows with closes distance in a different fam_id
    # dist_matrix = dist_matrix0.copy()
    features = df_original.drop('fam_id', axis=1)
    dist_matrix0 = pairwise_distances(features, metric='euclidean')

    dist_matrix = np.copy(dist_matrix0)
    # Reset the index of the DataFrame to keep track of original indices

    # OPTIMIZATION 1
    #############
    # WAS
    # # Get a boolean matrix where True represents rows with the same 'fam_id'.
    # same_fam_id = df_original['fam_id'].values[:, None] == df_original['fam_id'].values

    # # Set the distances for rows with the same 'fam_id' to infinity.
    # # np.inf is used here to ensure these distances are never considered as the minimum.
    # dist_matrix[same_fam_id] = np.inf

    # NOW..SIGNIFICANTLY FASTER
    # df_with_index = df_original.reset_index()

    # # Perform the merge to find all pairs with the same 'fam_id'
    # same_fam_id = df_with_index.merge(
    #     df_with_index[['index', 'fam_id']], on='fam_id', suffixes=('_left', '_right'))

    # num_rows = len(df_original)
    # # Initialize a boolean matrix of size (num_rows, num_rows)
    # same_fam_id_matrix = np.zeros((num_rows, num_rows), dtype=bool)

    # # Fill the boolean matrix with True where indices have the same 'fam_id'
    # same_fam_id_matrix[same_fam_id['index_left'].values, same_fam_id['index_right'].values] = True

    # dist_matrix[same_fam_id_matrix]= np.inf
    ##########

    # Get a boolean matrix where True represents rows with the same 'fam_id'.
    same_fam_id = df_original['fam_id'].values[:, None] == df_original['fam_id'].values

    # Set the distances for rows with the same 'fam_id' to infinity.
    # np.inf is used here to ensure these distances are never considered as the minimum.
    dist_matrix[same_fam_id] = np.inf

    # Find the index of the closest row and the distance for each row, ignoring those with the same 'fam_id'.
    closest_indices = dist_matrix.argmin(axis=1)
    closest_distances = dist_matrix.min(axis=1)

    # Add the results back to the original DataFrame.
    df_original['closest_row'] = closest_indices
    df_original['closest_distance'] = closest_distances

    # OPTIMIZATION 2
    #############
    # WAS

    # Assuming df_original is your original DataFrame and it includes the 'fam_id' column.

    # 2B set up families to migrate to and solve conflicts if mulitple row in the same family want to migrate to the different families
    # Add a column for the family that the closest row belongs to for easier reference
    # df_original['closest_family'] = df_original['closest_row'].apply(lambda x: df_original.at[x, 'fam_id'] if pd.notnull(x) else None)

    # # Determine the closest family for each original family based on the smallest distance
    # closest_family_for_group = df_original.groupby('fam_id').apply(lambda x: x.loc[x['closest_distance'].idxmin(), 'closest_family'])

    # # Map the target family for migration back to the original DataFrame
    # df_original['migrate_to_family'] = df_original['fam_id'].map(closest_family_for_group)

    # NOW

    df_original['closest_family'] = df_original.loc[df_original['closest_row'], 'fam_id'].values


    # Sort the DataFrame first by 'fam_id', then by 'closest_distance'
    sorted_df = df_original.sort_values(by=['fam_id', 'closest_distance'])

    # Drop duplicates, keeping the first entry for each 'fam_id'
    # which will be the one with the smallest 'closest_distance' due to the sort order
    result_df = sorted_df.drop_duplicates(subset='fam_id', keep='first')

    # Select only the 'fam_id' and 'closest_family' columns if necessary
    closest_family_for_group = result_df.set_index('fam_id')['closest_family']

    # Map the target family for migration back to the original DataFrame
    df_original['migrate_to_family'] = df_original['fam_id'].map(closest_family_for_group)

    ##################

    # OPTIMIZATION 3
    #############

    # WAS
    # Choosing only the closest family on the migrate_to_family side
    # Step 1: Determine the eligible family for each target family

    # Group by 'migrate_to_family' and find the family with the closest distance for each target
    # eligible_families = df_original.groupby('migrate_to_family').apply(
    #     lambda x: x.loc[x['closest_distance'].idxmin(), 'fam_id']
    # ).reset_index().rename(columns={0: 'eligible_fam_id'})

    # NOW
    # Sort the DataFrame by 'migrate_to_family' and 'closest_distance'
    sorted_df = df_original.sort_values(by=['migrate_to_family', 'closest_distance'])

    # Drop duplicates, keeping the first entry for each 'migrate_to_family'
    # which will be the one with the smallest 'closest_distance' due to the sort order
    unique_families = sorted_df.drop_duplicates(subset='migrate_to_family', keep='first')

    # Rename and clean up the result if necessary
    eligible_families = unique_families.rename(columns={'fam_id': 'eligible_fam_id'})[['migrate_to_family', 'eligible_fam_id']]

    ##############

    # Step 2: Assign migration permission based on eligibility
    # Merge the eligible families DataFrame back to the original DataFrame to assign migration permission
    df_original = df_original.merge(
        eligible_families,
        how='left',
        left_on=['migrate_to_family', 'fam_id'],
        right_on=['migrate_to_family', 'eligible_fam_id']
    )


    # Step 1 & 2: Group by 'migrate_to_family' and find all families with the smallest distance for each target
    # This involves creating a temporary DataFrame to calculate minimum distances for each migration target group
    min_distances = df_original.groupby('migrate_to_family')['closest_distance'].transform('min')

    # Step 3: Assign migration permission based on matching the minimum distance
    # Families that match the minimum distance to their target are allowed to migrate
    df_original['can_migrate'] = df_original['closest_distance'] == min_distances

    # Step 1: Find all fam_id values where at least one row has can_migrate = True
    fam_ids_to_update = df_original[df_original['can_migrate'] == True]['fam_id'].unique()

    # Step 2: Update can_migrate for rows where fam_id matches the identified fam_ids
    df_original.loc[df_original['fam_id'].isin(fam_ids_to_update), 'can_migrate'] = True

    # Logic to update 'migrate_to_family' where 'can_migrate' is False
    df_original.loc[df_original['can_migrate'] == False, 'migrate_to_family'] = df_original['fam_id']

    # Drop the 'eligible_fam_id' column as it's no longer needed
    df_original.drop('eligible_fam_id', axis=1, inplace=True)

    # Initialize NetworkX UnionFind



    ## OPTIMIZATION 4
    ##################################

    # WAS
    # # Using DataFrame.apply() to perform Union operations on all rows at once
    # uf = nx.utils.UnionFind()
    # df_original.apply(lambda row: uf.union(row['fam_id'], row['migrate_to_family']), axis=1)

    # # Mapping the results back to the DataFrame can be optimized by
    # # creating a mapping dictionary from the UnionFind structure and then using map()
    # fam_id_mapping = {item: uf[item] for item in uf.parents.keys()}

    # df_original['fam_id'] = df_original['fam_id'].map(fam_id_mapping)

    # Drop unnecessary columns in a more concise way
    # columns_to_drop = ['closest_row', 'closest_distance', 'closest_family', 'migrate_to_family', 'can_migrate']
    # df_original.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    # NOW
    # Iterate over DataFrame rows using itertuples() for better performance
    uf = nx.utils.UnionFind()
    for row in df_original.itertuples():
        uf.union(row.fam_id, row.migrate_to_family)

    # Creating a mapping dictionary from the UnionFind structure
    fam_id_mapping = {item: uf[item] for item in uf.parents.keys()}

    # Ensure that all original IDs are covered in the mapping
    df_original['fam_id'] = df_original['fam_id'].map(fam_id_mapping).fillna(df_original['fam_id'])

    # Drop unnecessary columns in a more concise way
    columns_to_drop = ['closest_row', 'closest_distance', 'closest_family', 'migrate_to_family', 'can_migrate']
    df_original.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    ####################################

    # # Calculate initial centroids
    centroids = calculate_centroids(df_original)

    # Reassign families based on closest centroid
    df_original = reassign_families(df_original, centroids)
    # df_original.reset_index(drop=True,inplace=True)

    # # Outlier removal


    # #Assuming df_original is your DataFrame and it already contains 'fam_id'
    # #Select numeric columns for centroid calculation, excluding 'fam_id' or any other non-relevant columns
    # features = [col for col in df_original.select_dtypes(include=np.number).columns if col != 'fam_id']

    # # Calculate the centroid for each family using a groupby operation, this remains efficient
    # centroids = df_original.groupby('fam_id')[features].transform('mean')

    # # Calculate pairwise distances to centroid for each member without using apply()
    # # Using scipy's cdist function for efficient distance calculation
    # distances_to_centroid = cdist(df_original[features], centroids, metric='euclidean').diagonal()
    # df_original['distance_to_centroid'] = distances_to_centroid

    # # Identify outliers using the IQR method for each family (only above upper bound), vectorized for efficiency
    # Q1 = df_original.groupby('fam_id')['distance_to_centroid'].transform(lambda x: x.quantile(0.25))
    # Q3 = df_original.groupby('fam_id')['distance_to_centroid'].transform(lambda x: x.quantile(0.75))
    # IQR = Q3 - Q1
    # upper_bound = Q3 + 1.5 * IQR

    # # Assigning -1 to fam_id for outliers, more directly
    # df_original.loc[df_original['distance_to_centroid'] > upper_bound, 'fam_id'] = -1

    # # Splitting outliers and non-outliers into separate DataFrames as needed
    # outliers_df = df_original[df_original['fam_id'] == -1].copy(deep=True)
    # df_original = df_original[df_original['fam_id'] != -1].copy(deep=True)

    # del(all_df_original)
    # all_df_original = pd.DataFrame()
    # df_original.drop('distance_to_centroid', axis=1, inplace=True)
    # outliers_df.drop('distance_to_centroid', axis=1, inplace=True)

    # # filtered_df1 = df_clean1.loc[df_original.index].copy()
    # # filtered_df1 = filtered_df1.join(df_original['fam_id'], how='left')
    # if ir == 0:
    #   filtered_df1 = df_clean1.join(df_original['fam_id'], how='right')
    #   filtered_outlier_df = df_clean1.loc[outliers_df.index]
    #   filtered_df1.reset_index(inplace=True, drop=True)
    # else:
    #   filtered_df2 = filtered_df1.join(df_original['fam_id'], how='right')
    #   filtered_outlier_df = filtered_df1.loc[outliers_df.index]
    #   filtered_outlier_df['fam_id'] = -1
    #   filtered_df2.reset_index(inplace=True, drop=True)
    #   filtered_df1 = filtered_df2.copy(deep=True)


    # all_df_original = pd.concat([all_df_original, filtered_df1], ignore_index=True)
    # all_outliers = pd.concat([all_outliers, filtered_outlier_df], ignore_index=True)
    # filtered_df1.drop('fam_id', inplace=True, axis=1)
    # # Optionally, drop the 'distance_to_centroid' column if no longer needed

    df_original.reset_index(inplace=True, drop=True)

    labels = df_original.fam_id
    selected_columns = df_original.drop(['fam_id'], axis = 1)
    sil_perf = silhouette_score(selected_columns, labels)
    calinski_harabasz = calinski_harabasz_score(selected_columns, labels)
    sil_perf_list.append(sil_perf)
    calinski_perf_list.append(calinski_harabasz)
    unique_fam_list.append(df_original.fam_id.nunique())
    iteration_list.append(ir)

    if df_original.fam_id.nunique() == breaker:
        break

data = {
    'iteration_list': iteration_list,
    'sil_perf_list': sil_perf_list,
    'calinski_perf_list': calinski_perf_list,
    'unique_fam_list': unique_fam_list
}

# Create DataFrame
df_eval = pd.DataFrame(data)

df_eval
#555

CPU times: total: 188 ms
Wall time: 382 ms


,iteration_list,sil_perf_list,calinski_perf_list,unique_fam_list
0,0,0.307999,394.834372,303
1,1,0.361714,287.235809,175
2,2,0.405565,256.649844,103
3,3,0.428583,297.048743,62
4,4,0.422243,281.018386,35
5,5,0.435867,323.781674,22
6,6,0.387908,460.719629,13
7,7,0.281347,326.963439,8
8,8,0.355716,275.212680,5
9,9,0.277275,336.132545,4


In [14]:
# Possible issue with all_outliers
df_comb1 = pd.concat([all_df_original, all_outliers], ignore_index=True)

In [18]:
df_comb1.to_excel('Godiva_true_umap_148.xlsx')

# Performance

## Test cluster performance


In [ ]:
# DSC- PURE
# df_original = df_original_2.copy()

### DOUBLE CHECK iloc
df_wine_selected_columns = df_original.drop('fam_id',axis=1).copy(deep=True)

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Assume 'labels' is the array of cluster labels obtained from a clustering algorithm
# For example: labels = [0, 1, 0, 1] - replace this with your actual labels
labels = df_original.fam_id
# Calculate metrics
silhouette_avg = silhouette_score(df_wine_selected_columns, labels)
calinski_harabasz = calinski_harabasz_score(df_wine_selected_columns, labels)
davies_bouldin = davies_bouldin_score(df_wine_selected_columns, labels)

print(f"Silhouette Score: {silhouette_avg}")
print(f"Calinski-Harabasz Index: {calinski_harabasz}")
print(f"Davies-Bouldin Index: {davies_bouldin}")

## For array

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Assuming 'new_array' is your NumPy array with the last column as 'fam_id'
# Drop 'fam_id' column for feature data
features = new_array[:, :-1].astype(float)
# features = new_array[:, :-1]  # All rows, all columns except the last

# Extract 'fam_id' for labels
labels = new_array[:, -1]  # Last column for labels

# Calculate metrics
silhouette_avg = silhouette_score(features, labels)
calinski_harabasz = calinski_harabasz_score(features, labels)
davies_bouldin = davies_bouldin_score(features, labels)

print(f"Silhouette Score: {silhouette_avg}")
print(f"Calinski-Harabasz Index: {calinski_harabasz}")
print(f"Davies-Bouldin Index: {davies_bouldin}")


## DSC Vis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import umap.umap_ as umap

# Assuming 'new_array' is a NumPy array and it's already loaded
# Prepare data: Assuming the first columns except the last are features since data is already standardized
features = new_array[:, :-1]  # All rows, all columns except the last
colors = new_array[:, -1]     # 'fam_id' is the last column

# Run UMAP
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
embedding = reducer.fit_transform(features)

# Plotting
plt.figure(figsize=(12, 10))
scatter = sns.scatterplot(
    x=embedding[:, 0],
    y=embedding[:, 1],
    hue=colors,
    palette="viridis",
    legend='full'
)
plt.title('UMAP projection of the dataset clusters, colored by fam_id')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend(title='fam_id')
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import umap.umap_ as umap
#import umap
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'first_row_each_group' is your DataFrame and it's already loaded
# new_array = new_array_2.copy()
# Prepare data: Assuming the first four columns are features since data is already standardized
features = new_array.drop('fam_id',axis=1).copy(deep=True)
#features = new_array.copy(deep=True)
# Assuming 'fam_id' is a column you want to use for coloring the points
colors = new_array['fam_id']

# Run UMAP
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
embedding = reducer.fit_transform(features)

# Plotting
plt.figure(figsize=(12, 10))
# Plot with hue based on 'fam_id'
scatter = sns.scatterplot(x=embedding[:, 0], y=embedding[:, 1], hue=colors, palette="viridis", legend='full')
#scatter = sns.scatterplot(x=embedding[:, 0], y=embedding[:, 1], palette="viridis", legend='full')
#Label points with the DataFrame index
# for i, point in enumerate(embedding):
#     plt.text(point[0]+0.1,  # Adding a small offset to x position for clarity
#              point[1],
#              new_array.index[i],  # The text is the index of the row
#              horizontalalignment='left',
#              size='small',
#              color='black')

plt.title('UMAP projection of the dataset clusters, colored by fam_id')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend(title='fam_id')
plt.show()
